In [ ]:
from skyfield.api import Topos, Star, load, utc
from datetime import datetime, timedelta
from skyfield.positionlib import position_of_radec
from skyfield.units import Angle
import numpy as np
import requests
import json
import time
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
import astropy.units as u
import numpy as np
import datetime

ra_hours = 220.90287878844615/ 15.0
dec_deg = 78.1016098453677
fov_tolerance_deg = 9.93

def find_best_location_time(ra_deg, dec_deg, fov_deg):
    # Setup sky coordinate from astrometry
    sky_coord = SkyCoord(ra=ra_deg*u.deg, dec=dec_deg*u.deg, frame='icrs')

    # Search range for lat/lon and time
    latitudes = np.arange(-90, 91, 10)  # every 10 degrees
    longitudes = np.arange(-180, 181, 10)

    # Try a few candidate times (e.g. a full night range on a fixed day)
    base_date = '2024-06-01'
    hours = np.arange(0, 24, 1)

    best_matches = []

    for lat in latitudes:
        for lon in longitudes:
            location = EarthLocation(lat=lat*u.deg, lon=lon*u.deg)
            for h in hours:
                t = Time(f"{base_date} {h:02d}:00:00")
                altaz = sky_coord.transform_to(AltAz(obstime=t, location=location))
                if altaz.alt.deg > 60:  # target is high in sky
                    best_matches.append((lat, lon, h, altaz.alt.deg))

    return sorted(best_matches, key=lambda x: -x[3])  # highest altitude first



def load_astrometry_data(filepath):
    """Load JSON result from astrometry.net"""
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def run_estimation(astrometry_json_path):
    print(f"Loading astrometry data from: {astrometry_json_path}")
    data = load_astrometry_data(astrometry_json_path)

    ra = data['ra']
    dec = data['dec']
    fov_deg = data['radius'] * 2  # radius to diameter
    orientation = data.get('orientation', None)

    print(f"Estimating observer location/time for:")
    print(f"  RA = {ra:.2f}°, Dec = {dec:.2f}°, FOV ~ {fov_deg:.2f}°")

    best_matches = find_best_location_time(ra, dec, fov_deg, orientation)

    print("\nTop 5 candidate location/time combos:")
    for i, match in enumerate(best_matches[:5]):
        print(f"{i+1}. Lat: {match['lat']}°, Lon: {match['lon']}°, "
              f"UTC Time: {match['utc_hour']:02d}:00, Altitude: {match['alt']:.1f}°")

if __name__ == "__main__":
    import sys
    if len(sys.argv) < 2:
        print("Usage: python estimate_sky_photo_origin.py <astrometry_json_file>")
    else:
        run_estimation(sys.argv[1])

Loading astrometry data from: --f=c:\Users\jainv\AppData\Roaming\jupyter\runtime\kernel-v3bb84894a480210e207049832add070f11c9ada47.json


OSError: [Errno 22] Invalid argument: '--f=c:\\Users\\jainv\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3bb84894a480210e207049832add070f11c9ada47.json'